#### Code to train classification models
**TODO**:
- verif dist fts
- GroupKFold ?
- inspect cat features
- catboost + xgboost + lgbm
- NaNs ?

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ../src

## Imports

In [3]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = "1"

import torch
torch.cuda.get_device_name(0)

'NVIDIA RTX A6000'

In [4]:
import os
import gc
import re
import glob
import json
import cudf
import lofo
import torch
import pickle
import optuna
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from copy import deepcopy
from tqdm.notebook import tqdm
from collections import Counter
from pandarallel import pandarallel
from numerize.numerize import numerize
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GroupKFold
from cuml.feature_extraction.text import TfidfVectorizer

pandarallel.initialize(progress_bar=False, use_memory_fs=False)
pd.options.display.max_columns = 500
pd.options.display.max_rows = 200
warnings.simplefilter("ignore")

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [5]:
from pp import *
from dtypes import *
from params import *
from utils.metrics import *
from inference.main import k_fold_inf
from data.preparation import reduce_mem_usage
from matching import get_CV, load_cleaned_data, cci

### Params

In [6]:
N_FOLDS = 5
THRESHOLD = 0.0075

## Data


In [7]:
df = pd.read_csv(DATA_PATH + "train.csv")[["id", "point_of_interest"]]

In [8]:
LEVEL = 2

In [9]:
# df_p = pd.read_csv(OUT_PATH + f"features_train_{LEVEL}.csv")

df_p = pd.read_csv(OUT_PATH + f"features_train_{LEVEL}_{THRESHOLD}.csv", dtype=DTYPES_2)

In [10]:
df_p = df_p.sort_values(['id_1', 'id_2']).reset_index(drop=True)

## Exp

In [15]:
EXP_FOLDER = LOG_PATH + "lvl_2/" + "2022-06-28/0/"  # lgb 10kf - 0.8982


EXP_FOLDER = LOG_PATH + "lvl_2/" + "2022-06-30/5/"  # lgb gkf - 0.8837
# EXP_FOLDER = LOG_PATH + "lvl_2/" + "2022-06-30/6/"  # xgb gkf - 0.8782

EXP_FOLDERS = [
    LOG_PATH + "lvl_2/" + "2022-06-30/5/",  # lgb gkf - 0.8837
    LOG_PATH + "lvl_2/" + "2022-07-01/2/",   # xgb gkf - 0.8818
]

# EXP_FOLDERS = [
#     LOG_PATH + "lvl_2/" + "2022-07-01/1/",  # lgb 10kf -
#     LOG_PATH + "lvl_2/" + "2022-07-01/0/",  # xgb 10kf -
# ]

WEIGHTS = [0.75, 0.25]
# WEIGHTS = [1, 0]

In [17]:
# pred_oof = np.load(EXP_FOLDER + "pred_oof.npy")

pred_oof_ = np.average([
    np.load(f + "pred_oof.npy") for f in EXP_FOLDERS
], weights=WEIGHTS, axis=0)

In [1]:
preds_lgbm = k_fold_inf(
    df_p,
    EXP_FOLDERS[0],
)

preds_xgb = k_fold_inf(
    df_p,
    EXP_FOLDERS[1],
)

pred_oof = WEIGHTS[0] * preds_lgbm + WEIGHTS[1] * preds_xgb

NameError: name 'k_fold_inf' is not defined

In [92]:
roc_auc_score(df_p['match'], pred_oof)

0.9508260754268621

In [87]:
roc_auc_score(df_p['match'], pred_oof)

0.9507679122799162

### Restrict to a subset comparable to LB

In [83]:
# SUBSET = [0, 1, 2]

# df = pd.read_csv(f'../output/folds_5.csv')
# df = df[df['fold'].isin(SUBSET)]

# pred_oof = pred_oof[(df_p['fold_1'].isin(SUBSET)) & (df_p['fold_2'].isin(SUBSET))]

# df_p = df_p[(df_p['fold_1'].isin(SUBSET)) & (df_p['fold_2'].isin(SUBSET))].reset_index(drop=True)

In [84]:
roc_auc_score(df_p['match'], pred_oof)

0.9508480425294981

### Add m_true column

In [77]:
if "m_true" not in df.columns:
    df = df.reset_index()
    df = df.sort_values(by=["point_of_interest", "id"]).reset_index(drop=True)

    id_all = np.array(df["id"])
    poi_all = np.array(df["point_of_interest"])
    poi0 = poi_all[0]
    id0 = id_all[0]

    di_poi = {}
    for i in range(1, df.shape[0]):
        if poi_all[i] == poi0:
            id0 = str(id0) + " " + str(id_all[i])
        else:
            di_poi[poi0] = str(id0) + " "  # need to have trailing space in m_true
            poi0 = poi_all[i]
            id0 = id_all[i]

    di_poi[poi0] = str(id0) + " "  # need to have trailing space in m_true
    df["m_true"] = df["point_of_interest"].map(di_poi)

    df = df.sort_values(by="index").reset_index(
        drop=True
    )  # sort back to original order
    df.drop("index", axis=1, inplace=True)

In [78]:
THRESHOLDS = [0.45, 0.6, 0.6, 0.9, 0.]  # FULL DATA

# THRESHOLDS = [0.3, 0.45, 0.45, 0.8, 0]  # RESTRICTED  - cuts = [0.37, 0.48, 0.43, 0.77]
# THRESHOLDS = [0.5, 0.5, 0.5, 0.8]

# THRESHOLDS = [0.37, 0.48, 0.43, 0.77, 0.]  # LB / RESTRICTED

threshold, threshold_small, threshold_big, threshold_merge_max, threshold_merge_avg = THRESHOLDS

In [90]:
id_to_poi, poi_to_id, preds = get_improved_CV(df_p, pred_oof, df, THRESHOLDS)

CV 0.8823


ValueError: not enough values to unpack (expected 5, got 3)

In [79]:
id_to_poi, poi_to_id, poi_counts, preds, cv = get_improved_CV(df_p, pred_oof, df, THRESHOLDS)

CV 0.8843


## PP

### Prepare

In [90]:
df2 = df_p.copy()
df2["match"] = np.copy(pred_oof) #.astype("float32")

try:
    df2 = df2[['id_1', 'id_2', "match"]]
    df2.columns = ['id', 'id2', "match"]
except KeyError:
    df2 = df2[['id', 'id2', "match"]]

# sort by decr prediction
df2 = df2.sort_values(by=["match"], ascending=False).reset_index(drop=True)

### Construct POI from pairs

In [91]:
id_to_poi, poi_to_id, poi_counts = match_pois(
    df2,
    threshold=threshold,
    threshold_small=threshold_small,
    threshold_big=threshold_big
)

### Merge POIs

#### Original

In [92]:
id_to_poi, poi_to_id, poi_counts = merge_pois_simple(
    df2,
    id_to_poi,
    poi_to_id,
    poi_counts,
    threshold=threshold,
    threshold_merge=threshold_merge_max,
)

#### Theo

In [93]:
# id_to_poi, poi_to_id, poi_counts = merge_pois_advanced(
#     df2,
#     id_to_poi,
#     poi_to_id,
#     poi_counts,
#     threshold_merge_avg=threshold_merge_avg,
#     threshold_merge_max=threshold_merge_max
# )

In [94]:
id_to_poi_svg = deepcopy(id_to_poi)

#### Remove outliers
- TODO : add lvl 1 as nomatch

In [98]:
id_to_poi = deepcopy(id_to_poi_svg)

poi_to_id = get_poi_to_id(id_to_poi)

In [100]:
# THRESHOLD_OUT = 0.3
THRESHOLD_OUT = 0.25  # sampled data

In [101]:
id_to_poi, poi_to_id = remove_outliers(df2, id_to_poi, poi_to_id, threshold_out=THRESHOLD_OUT)

In [102]:
# len(removed)  # TODO : retrieve matches for removed ??

In [103]:
# poi_to_id = {}
# for k in id_to_poi.keys():
#     if id_to_poi[k] not in poi_to_id:
#         poi_to_id[id_to_poi[k]] = []
#     poi_to_id[id_to_poi[k]].append(k)

### CV

In [104]:
preds = pd.DataFrame.from_dict(id_to_poi, orient="index").reset_index()
preds['matches'] = preds[0].map(poi_to_id).apply(lambda x: " ".join(x))
preds.columns = ["id", "poi", "m2"]

In [105]:
cv = evaluate(df, preds)
print(f"CV {cv:.5f}")

CV 0.90380


In [97]:
cv = evaluate(df, preds)
print(f"CV {cv:.5f}")

CV 0.90410


- base : 0.90228
- theo : 0.90312
- theo - outlier : 0.90405

- base : 0.9192
- theo : 0.91975
- theo - outlier : 0.9205

## Inference 2a

In [106]:
df2 = df_p.copy()
df2["match"] = np.copy(pred_oof) #.astype("float32")

try:
    df2 = df2[['id_1', 'id_2', "match"]]
    df2.columns = ['id', 'id2', "match"]
except KeyError:
    df2 = df2[['id', 'id2', "match"]]

# sort by decr prediction
df2 = df2.sort_values(by=["match"], ascending=False).reset_index(drop=True)

In [136]:
# THS = [0.45, 0.6, 0.6, 0.9]
THS = [0.5, 0.6, 0.6, 0.8, 0]

THS = [0.2, 0.2, 0.2, 0.2, 0]

# THS = [0.37, 0.48, 0.43, 0.77, 0.]

In [137]:
id_to_poi, poi_to_id, poi_counts, preds, cv = get_improved_CV(df_p, pred_oof, df, THS, max_size=np.inf)

CV 0.8624


### Rebuild pairs

In [138]:
# construct all possible pairs for each poi
p1 = []
p2 = []

for poi in poi_to_id.keys():
    ids = poi_to_id[poi]
    ids.sort()
    step = max(1, int(0.5 + len(ids) ** 2 / 2 / 2000)) # limit to approx 2000 per poi
    for i in range(len(ids) - 1):
        for j in range(i + 1, len(ids), step):
            p1.append(ids[i])
            p2.append(ids[j])

p1 = pd.DataFrame(p1)
p1.columns = ['id']
p2 = pd.DataFrame(p2)
p2.columns = ['id']

In [139]:
# remove duplicate pairs
p12 = pd.concat([p1['id'], p2['id']], axis=1)
p12.columns = ['id','id2']
p12 = p12.reset_index()

# flip - only keep one of the flipped pairs, the other one is truly redundant
idx = p12['id'] > p12['id2']
p12['t'] = p12['id']
p12['id'].loc[idx] = p12['id2'].loc[idx]
p12['id2'].loc[idx] = p12['t'].loc[idx]

p12 = p12.sort_values(by=['id', 'id2']).reset_index(drop=True)
p12 = p12.drop_duplicates(subset=['id', 'id2'])

# also drop id == id2 - it may happen
p12 = p12.loc[p12['id'] != p12['id2']]
p1 = p1.loc[p12['index']].reset_index(drop=True)
p2 = p2.loc[p12['index']].reset_index(drop=True)

In [140]:
# Keep only new pairs
new_pairs = pd.concat([p1, p2], 1)
new_pairs.columns = ["id", "id2"]

new_pairs = new_pairs.merge(df2, how="left")
new_pairs = new_pairs[new_pairs["match"].isna()].drop("match", axis=1).reset_index()

p1 = new_pairs[["id"]]
p2 = new_pairs[["id2"]]
p2.columns = ['id']

In [141]:
print(f"Added {len(p1)} pairs")

Added 340292 pairs


### FE

In [142]:
# add other columns needed for FE
cols = [
    "id",
    "name",
    "latitude",
    "longitude",
    "address",
    "country",
    "url",
    "phone",
    "city",
    "categories",
    "category_simpl",
    "categories_split",
    "cat2",
    "idx",
    "state",
    "zip",
]
train = load_cleaned_data(OUT_PATH + "cleaned_data_train.csv")
train["idx"] = np.arange(len(train))

p1 = p1[["id"]].merge(train[cols], on="id", how="left")
p2 = p2[["id"]].merge(train[cols], on="id", how="left")

In [143]:
from fe_main import feature_engineering_1, feature_engineering_2

In [144]:
df_p3 = feature_engineering_1(p1, p2, train.copy(), ressources_path=RESSOURCES_PATH)

- Distances
- Features for column : name
- Features for column : categories
- Features for column : address
- Nan features
- Matching
- Category match
- Ratios
- Count encodings
- Computing position distances
- Computing feature same_state
- Computing feature same_zip
- Computing feature same_city
- Column : name  -  Function : levenshtein
- Column : address  -  Function : levenshtein
- Column : url  -  Function : levenshtein


In [145]:
df_p3 = feature_engineering_2(df_p3, train.copy(), ressources_path=RESSOURCES_PATH)

- Cat links & quantiles
- Simply connected components
- Strongly connected components
- Cat link score
- Link between grouped categories
- Features for column name_initial
- Features for column name_initial_decode
- Features for column nameC
- Features for column name
- Features for column categories
- Features for column address
- Features for column url
- Features for column city
- Features for column state
- Features for column zip
- Features for column phone
- Count encoding
- Words in categories
- Words in names
- Close count
- Close count of same category2
- Close count of same category_simpl
- Compare numeric part of the name/address
- Number in names features
- Computing nan features
- Computing feature name_tf_idf_33_char_wb_sim
- Computing feature address_tf_idf_33_char_wb_sim
- Computing feature url_tf_idf_33_char_wb_sim
- Column : name  -  Function : wratio
- Column : name  -  Function : partial_ratio
- Column : address  -  Function : wratio
- Column : address  -  Function 

In [146]:
N_FOLDS = 5

path = f'../output/folds_{N_FOLDS}.csv'
df_split = pd.read_csv(path)

In [147]:
if "fold_1" not in df_p3.columns:
    df_p3 = df_p3.merge(df_split, left_on="id_1", right_on="id", how="left").drop("id", axis=1)
    df_p3 = df_p3.merge(df_split, left_on="id_2", right_on="id", how="left", suffixes=('_1', '_2')).drop("id", axis=1)

In [148]:
if "match" not in df_p3.columns:
    df_p3['match'] = (df_p3['point_of_interest_1'] == df_p3['point_of_interest_2']).astype(int)

### Inference

In [150]:
preds_lgbm = k_fold_inf(
    df_p3,
    EXP_FOLDERS[0],
)

preds_xgb = k_fold_inf(
    df_p3,
    EXP_FOLDERS[1],
)

preds_2 = WEIGHTS[0] * preds_lgbm + WEIGHTS[1] * preds_xgb


-------------   Fold 1 / 5  -------------

- Scoring 172276 pairs

[W] [10:56:43.196391] Treelite currently does not support float64 model parameters. Accuracy may degrade slightly relative to native LightGBM invocation.
[W] [10:56:43.693205] Casting all thresholds and leaf values to float32, as FIL currently doesn't support inferencing models with float64 values. This may lead to predictions with reduced accuracy.
- AUC = 0.9237

-------------   Fold 2 / 5  -------------

- Scoring 163185 pairs

[W] [10:56:44.225203] Treelite currently does not support float64 model parameters. Accuracy may degrade slightly relative to native LightGBM invocation.
[W] [10:56:44.644870] Casting all thresholds and leaf values to float32, as FIL currently doesn't support inferencing models with float64 values. This may lead to predictions with reduced accuracy.
- AUC = 0.9119

-------------   Fold 3 / 5  -------------

- Scoring 174615 pairs

[W] [10:56:45.052487] Treelite currently does not support floa

In [151]:
df3 = df_p3[["id_1", "id_2"]].copy()
df3["match"] = preds_2

df3.columns = ["id", "id2", "match"]

## Final PP
- Could work with my approach

In [152]:
df_all = pd.concat([df2, df3], 0, ignore_index=True)

In [129]:
THRESHOLDS = [0.5, 0.6, 0.6, 0.8, 0]
# THRESHOLDS = [0.45, 0.6, 0.6, 0.9]

In [161]:
THRESHOLDS = [0.37, 0.48, 0.43, 0.77, 0.5]

In [162]:
_ = get_improved_CV(df_all, df_all["match"], df, THRESHOLDS)

CV 0.9007


In [135]:
_ = get_improved_CV(df2, df2["match"], df, THRESHOLDS)

CV 0.9041
